In [125]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 4.3 MB/s 
     |████████████████████████████████| 106 kB 43.5 MB/s 
     |████████████████████████████████| 278 kB 44.9 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 213 kB 65.8 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 2.3 MB 29.1 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 80 kB 7.3 MB/s 
     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 68 kB 5.4 MB/s 
     |████████████████████████████████| 68 kB 5.6 MB/s 
     |████████████████████████████████| 46 kB 3.0 M

In [126]:
import pandas as pd
import re
from nltk import flatten
import numpy as np
import pandas as pd
import numpy as np
import torch
import torchtext
from torchtext import data
import torch.optim as optim
import tensorflow as tf
import argparse
import os
import torch.nn.functional as F
import torch.nn as nn
import logging
import matplotlib.pyplot as plt

import gradio as gr
np.random.seed(43)
torch.manual_seed(43)

In [2]:
glove=torchtext.vocab.GloVe(name='840B',dim=300)

In [3]:
class Baseline(nn.Module):#Baseline model

    def __init__(self, embedding_dim, vocab):
        super(Baseline, self).__init__()
        #100 vectors for each word 
        #embedding.shape=[embedding dim,glove.len]
        self.embedding = nn.Embedding.from_pretrained(vocab.vectors)
        self.fc = nn.Linear(embedding_dim, 60)

    def forward(self, x, len=None):
        #x has shape [sentence length, batch size]
        embedded = self.embedding(x)#embedded.shape=[sentence len, batch size,embedding size]
        # print("embedded",embedded.shape)
        average = embedded.mean(0) # average.shape=[batch size, embedding size]
        # print("average",average.shape)
        prediction = self.fc(average)
        # print("prediction",prediction.shape)
        # prediction=torch.nn.Softmax()(prediction)
        #prediction=F.log_softmax(prediction,dim=1)
        # print("lof sofamax",prediction.shape)
        return prediction

In [4]:
model = Baseline(300,glove)
model.load_state_dict(torch.load('/content/drive/MyDrive/AAA_1786_model_baseline.pt',map_location=torch.device('cpu')))
model.eval()

Baseline(
  (embedding): Embedding(2196017, 300)
  (fc): Linear(in_features=300, out_features=60, bias=True)
)

In [118]:
emotion_score = pd.read_csv('emotion.csv')
display(emotion_score)

,Unnamed: 0,emoji,emoji_num,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,0,👏,0,0.08,0.39,0.08,0.00,0.64,0.00,0.25,0.53
1,1,💖,1,0.00,0.33,0.00,0.00,0.89,0.00,0.25,0.53
2,2,🌹,2,0.00,0.36,0.00,0.00,0.56,0.00,0.11,0.72
3,3,✅,3,0.00,0.31,0.00,0.00,0.19,0.00,0.00,0.33
4,4,😔,4,0.25,0.22,0.31,0.28,0.00,0.72,0.19,0.14
5,5,💗,5,0.00,0.36,0.00,0.00,0.89,0.00,0.22,0.50
6,6,⬅️,6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,7,💙,7,0.00,0.25,0.00,0.00,0.61,0.17,0.17,0.28
8,8,😄,8,0.00,0.36,0.00,0.00,0.86,0.00,0.28,0.28
9,9,😘,9,0.00,0.33,0.00,0.00,0.72,0.00,0.17,0.61


In [6]:
emoji_dict={}
keys = range(60)
values = emotion_score['emoji']
for i in keys:
  emoji_dict[i]=values[i]
print(emoji_dict)

{0: '👏', 1: '💖', 2: '🌹', 3: '✅', 4: '😔', 5: '💗', 6: '⬅️', 7: '💙', 8: '😄', 9: '😘', 10: '😳', 11: '😋', 12: '🙏', 13: '👌', 14: '✨', 15: '😞', 16: '💕', 17: '💜', 18: '💀', 19: '😌', 20: '😹', 21: '⚡️', 22: '✔️', 23: '👅', 24: '😚', 25: '😓', 26: '😠', 27: '😣', 28: '😬', 29: '☕️', 30: '➖', 31: '🎧', 32: '💓', 33: '💰', 34: '💘', 35: '😮', 36: '😻', 37: '👇', 38: '✊', 39: '🔴', 40: '💛', 41: '🌷', 42: '😡', 43: '💩', 44: '💋', 45: '🌈', 46: '💚', 47: '❗️', 48: '📍', 49: '🎁', 50: '👑', 51: '😝', 52: '☁️', 53: '🌺', 54: '😫', 55: '😕', 56: '😃', 57: '💃', 58: '🍃', 59: '▶️'}


In [111]:
emotion_score[user_input_2].to_numpy().sum()

21.23

Please input a sentence:The flower is so pretty
Please input your emotion:joy


In [153]:
user_input=input('Please input a sentence:')
#anger, anticipation, disgust, fear, joy, sadness, surprise and trust
user_input_2=input('Please input your emotion:')

df=pd.DataFrame({'user input:',user_input})
L= re.sub('[0-9]+','',user_input)
L=re.split('[ ,.!?:$&*/\|<>^+=-_-•%]',L)
L=[j for j in L if j]
user_input_token=torch.tensor([glove.stoi.get(w, len(glove.stoi)-1) for w in L])

prediction = model(user_input_token)
prob,pre=torch.topk(F.softmax(prediction),15,dim=0)

prob=prob.tolist()
new_pred_dict={}
j=0
sum1=emotion_score[user_input_2].to_numpy().sum()
#print(emotion_score[user_input_2])
#list1=F.softmax(torch.from_numpy(emotion_score[user_input_2].to_numpy()),dim=0)
#print(list1)
for i in pre.tolist():
  #print(emoji_dict[i])
  
  final_score=0.5*prob[j]+0.5*emotion_score[user_input_2][i]/sum1
  print(emoji_dict[i],'prob=',prob[j],'emotion score=',emotion_score[user_input_2][i],final_score)
  new_pred_dict[emoji_dict[i]]=final_score
  #print(final_score)
  j=j+1
print(new_pred_dict)
sorted_emoji_score= sorted(new_pred_dict.items(), key=lambda x:x[1],reverse=True)
print(sorted_emoji_score)

print('Input is:',user_input)
print('emotion is:',user_input_2)
print('Predicted Top 2 Emoji:',[i[0] for i in sorted_emoji_score[:2]])

Please input a sentence:It's rainy outside
Please input your emotion:joy
😃 prob= 0.196254700422287 emotion score= 0.83 0.11767515991439362
🍃 prob= 0.11910435557365417 emotion score= 0.11 0.06214285136195662
✊ prob= 0.07094167172908783 emotion score= 0.03 0.036177383203215605
⚡️ prob= 0.05440353602170944 emotion score= 0.0 0.02720176801085472
👌 prob= 0.043775524944067 emotion score= 0.22 0.02706910962229257
😡 prob= 0.03933308646082878 emotion score= 0.0 0.01966654323041439
💜 prob= 0.037102073431015015 emotion score= 0.47 0.029620278354697334
✅ prob= 0.03687480837106705 emotion score= 0.19 0.0229122039971209
💘 prob= 0.03636281192302704 emotion score= 0.67 0.03396096319184795
💚 prob= 0.031052976846694946 emotion score= 0.58 0.029186403637666832
🎁 prob= 0.026654629036784172 emotion score= 0.0 0.013327314518392086
😌 prob= 0.02476394549012184 emotion score= 0.81 0.03145875088919658
💕 prob= 0.01712527498602867 emotion score= 0.83 0.028110447196264453
😣 prob= 0.016993695870041847 emotion score

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [152]:
def baseline_emoji(user_input,emotion):
  #seperate the input sentence and tokenize
  L= re.sub('[0-9]+','',user_input)
  L=re.split('[ ,.!?:$&*/\|<>^+=-_-•%]',L)
  L=[j for j in L if j]
  user_input_token=torch.tensor([glove.stoi.get(w, len(glove.stoi)-1) for w in L])
  prediction = model(user_input_token)
  #find probability
  prob,pre=torch.topk(F.softmax(prediction),15,dim=0)
  prob=prob.tolist()
  
  if emotion == 'none':#no emotion
    #print([emoji_dict[i] for i in pre[:3]])
    return [emoji_dict[i] for i in pre[:3]]
  else:#with emotion
    new_pred_dict={}#create dictionary to store the score
    j=0
    sum1=emotion_score[user_input_2].to_numpy().sum()
    for i in pre.tolist():
      #print(emoji_dict[i])
      final_score=0.5*prob[j]+0.5*emotion_score[user_input_2][i]/sum1
      #print(emoji_dict[i],'prob=',prob[j],'emotion score=',emotion_score[user_input_2][i],final_score)
      new_pred_dict[emoji_dict[i]]=final_score
      #print(final_score)
      j=j+1
    #print(new_pred_dict)
    sorted_emoji_score= sorted(new_pred_dict.items(), key=lambda x:x[1],reverse=True)
    #print(sorted_emoji_score)
  return [i[0] for i in sorted_emoji_score[:3]]

In [151]:
def change_textbox(choice):#anger, anticipation, disgust, fear, joy, sadness, surprise and trust
    if choice == "anger":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "anticipation":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "disgust":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "fear":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "joy":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "sadness":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "surprise":
        return gr.Textbox.update(lines=8, visible=True)
    elif choice == "trust":
        return gr.Textbox.update(lines=8, visible=True)
    else:
        return gr.Textbox.update(visible=False)


with gr.Blocks() as block:
    InputSentence = gr.Textbox(lines=2, interactive=True,label='Input your sentence here:')
    radio = gr.Radio(["anger", "anticipation", "disgust", "fear", "joy", "sadness", "surprise","trust", "none"], label="What's your emotion")
    Emoji = gr.Textbox(lines=2, interactive=True,label='Your emoji:')
    radio.change(fn=change_textbox, inputs=radio)
    b1 = gr.Button("Go Emoji!")
    #b2 = gr.Button("Go Emoji!")
    b1.click(fn=baseline_emoji, inputs=[InputSentence,radio], outputs=Emoji)
block.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [138]:
demo=gr.Interface(fn=baseline_emoji,inputs=['text','text'],outputs=['text'])
demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>